# Gather Historical ETF Data 

In [127]:
import os
from datetime import datetime, timedelta

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
from yahoofinancials import YahooFinancials

## Data Download

In [93]:
ticker_data_filepath = 'ticker_data_1m.csv'

tickers = 'XBB.TO XINC.TO XCNS.TO XBAL.TO XGRO.TO XEQT.TO VAB.TO VCIP.TO VCNS.TO VBAL.TO VGRO.TO VEQT.TO'
result_dfs = []

today = datetime.now()
increment_size = timedelta(days=6)   # maximum number of days allowed per query for minute-by-minute data

In [94]:
# minute-by-minute data is only available within the last 30 days
start_date = today - timedelta(days=29)

# initialize ticker data storage
ticker_df = pd.DataFrame()

# iterate from inception date to today. yfinance takes care of end_date conditions if we go past current day.
while start_date < today:
    end_date = start_date + increment_size
    ticker_df = ticker_df.append(yf.download(tickers=tickers, 
                                             start=start_date.strftime('%Y-%m-%d'),
                                             end=end_date.strftime('%Y-%m-%d'),
                                             interval='1m',
                                             progress=False))
    start_date = end_date + timedelta(days=1)

In [95]:
# all columns are the same with a 1 minute time interval - only need to keep one
ticker_df = ticker_df['Adj Close']
ticker_df = ticker_df.rename(columns={'Adj Close': 'Price'})

# persist data
ticker_df.to_csv(ticker_data_filepath, mode='a')

In [118]:
# load all gathered data for analysis
full_ticker_df = pd.read_csv(ticker_data_filepath)

# drop duplicates from the same minute
full_ticker_df.drop_duplicates(subset='Datetime', inplace=True)

In [119]:
# extract date information from index for easier analysis
def extract_time(string_timestamp):
    if ":" == string_timestamp[-3:-2]:
        string_timestamp = string_timestamp[:-3] + string_timestamp[-2:]
    timestamp = datetime.strptime(string_timestamp, "%Y-%m-%d %H:%M:%S%z")
    return pd.Series((timestamp, timestamp.strftime('%H%M'), timestamp.strftime('%w')))

full_ticker_df[['Datetime', 'time_of_day', 'day_of_week']] = full_ticker_df["Datetime"].apply(lambda timestamp: extract_time(timestamp))

## Find Peaks and Troughs of Daily Prices
TODO

## Find Peaks and Troughs of Weekly Prices
TODO

### Useful Tidbits

In [ ]:
# fetch inception date of ticker
# inception_date  = yf.download(tickers=ticker, period='max').index[0]